In [9]:
import tensorflow as tf 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [10]:
train = pd.read_csv("D:\Mini Project\minipj\data\Suicide_Detection.csv")
data = []
max_len= 0
max_len_seq = 0 
for i in train['text']:
    data.append(i)
    max_len+=len(i)
    max_len_seq= max(max_len_seq,len(i))
train.loc[train['class']== 'suicide'] = 1
train.loc[train['class']== 'non-suicide'] = 0
pred = np.asarray(train['class'])


In [11]:
txt, x_test, acc_pred, y_test = train_test_split(data, pred, test_size=0.10, random_state=0)
txt, x_val, acc_pred, y_val= train_test_split(txt, acc_pred, test_size=0.20, random_state=0)

In [12]:
tokenizer = Tokenizer(num_words = 16004, oov_token='<OOV>')
tokenizer.fit_on_texts(txt)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(txt)
padded = pad_sequences(sequences, padding='post', truncating='post', maxlen= 500)
tf.config.list_physical_devices('GPU')
val_sequences = tokenizer.texts_to_sequences(x_val)
val_padded = pad_sequences(val_sequences, padding='post', truncating='post', maxlen= 500)
test_sequences = tokenizer.texts_to_sequences(x_test)
test_padded = pad_sequences(test_sequences, padding='post', truncating='post', maxlen= 500)
training_padded = np.array(padded).astype(int)
training_labels = np.array(acc_pred).astype(int)
val_padded = np.array(val_padded).astype(int)
val_labels = np.array(y_val).astype(int)
test_padded = np.array(test_padded).astype(int)
test_labels = np.array(y_test).astype(int)
training_labels = training_labels.reshape(-1,1)

In [16]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(16004,8,input_shape =training_padded[0].shape),
    tf.keras.layers.LSTM(28,return_sequences=True),
    tf.keras.layers.LSTM(28),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

num_epochs = 10
history = model.fit(training_padded, training_labels, epochs=num_epochs,  validation_data=(val_padded, val_labels), verbose=1)

Epoch 1/10
5222/5222 [==============================] - 193s 36ms/step - loss: 0.4941 - accuracy: 0.7107 - val_loss: 0.2129 - val_accuracy: 0.9219
Epoch 2/10
5222/5222 [==============================] - 186s 36ms/step - loss: 0.1810 - accuracy: 0.9346 - val_loss: 0.1586 - val_accuracy: 0.9426
Epoch 3/10
5222/5222 [==============================] - 186s 36ms/step - loss: 0.1407 - accuracy: 0.9494 - val_loss: 0.1605 - val_accuracy: 0.9398
Epoch 4/10
5222/5222 [==============================] - 190s 36ms/step - loss: 0.1206 - accuracy: 0.9571 - val_loss: 0.1463 - val_accuracy: 0.9471
Epoch 5/10
5222/5222 [==============================] - 191s 37ms/step - loss: 0.1093 - accuracy: 0.9607 - val_loss: 0.1384 - val_accuracy: 0.9483
Epoch 6/10
5222/5222 [==============================] - 190s 36ms/step - loss: 0.0988 - accuracy: 0.9650 - val_loss: 0.1369 - val_accuracy: 0.9502
Epoch 7/10
5222/5222 [==============================] - 191s 37ms/step - loss: 0.0906 - accuracy: 0.9678 - val_loss: 0

In [16]:
tf.__version__

'2.6.0'

In [22]:
loaded_model = model.load_model("D:\Mini Project\minipj\models\high_prec.h5")

AttributeError: 'Sequential' object has no attribute 'load_model'

In [20]:
y_pred = model.predict(test_padded)
y_pred[y_pred>=0.5]= 1
y_pred[y_pred<0.5]= 0
from sklearn.metrics import accuracy_score, precision_score
print((accuracy_score(test_labels,y_pred))*100)
print((precision_score(test_labels, y_pred)))

93.89434677697346
0.9191720059142434


In [1]:
output = model.layers[-2].output
model_features = tf.keras.Model(inputs=model.input, outputs=output)

train_features = model_features.predict(training_padded)
val_features = model_features.predict(val_padded)
test_features = model_features.predict(test_padded)


NameError: name 'model' is not defined

In [32]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dense
from keras.models import Sequential
from gensim.models import FastText  # Adjust if using a different FastText library
import fasttext.util
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from keras.regularizers import l1



# Load and preprocess text data
data = pd.read_excel("D:\Mini Project\minipj\data\Suicide_Detection_cleaned.xlsx")
data = data.dropna()
X_train, X_test, y_train, y_test = train_test_split(data["text"], data["class"], test_size=0.25) # Replace with your data splitting function

# Tokenize text (if not already pre-tokenized)
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Pad sequences to a uniform length
max_length = 100  # Adjust as needed
X_train_padded = tf.keras.preprocessing.sequence.pad_sequences(X_train_sequences, maxlen=max_length)
X_test_padded = tf.keras.preprocessing.sequence.pad_sequences(X_test_sequences, maxlen=max_length)

# Load or create FastText model
fasttext_model = fasttext.load_model("D:\Mini Project\minipj\models\cc.en.300.bin")
vocab_size = len(tokenizer.word_index) + 1  # Account for unknown words
embedding_dim = 300  # Match FastText embedding dimension
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < vocab_size:
        embedding_vector = fasttext_model.get_word_vector(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

#label encoding
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

# Define CNN-BiLSTM model
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length, weights=[embedding_matrix], trainable=False),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=2),
    Bidirectional(LSTM(64)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train, epochs=10, batch_size=32, validation_data=(X_test_padded, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test_padded, y_test)
print("Test accuracy:", accuracy)


ValueError: With n_samples=1, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [29]:
!pip install fasttext

  Using cached fasttext-0.9.2.tar.gz (68 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pybind11-2.11.1-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp39-cp39-win_amd64.whl size=233232 sha256=bba612de03551dcecf074f8e96a5ae55bb8f7fc721ef2dabe3ba55930ac76dee
  Stored in directory: c:\users\rosha\appdata\local\pip\cache\wheels\64\57\bc\1741406019061d5664914b070bd3e71f6244648732bc96109e
Successfully built fasttext
